# Evaluation des modèles sur un nouveau jeu de données
## Chris HOZE Novembre 2024

### Visualisation du nouveau dataset et selection des données
On utilise les données COVID_NET accessibles ici :
https://www.kaggle.com/datasets/andyczhao/covidx-cxr2/data


In [1]:
import pandas as pd
import numpy as np
import os
import shutil

In [2]:
# Import des tableaux

path = r"C:\Users\Inrae\Documents\Projet_Data_Science\COVID_NET"

train_data = pd.read_csv(os.path.join(path,"train.txt"), sep=" ", header = None, index_col = 0)
test_data = pd.read_csv(os.path.join(path,"test.txt"), sep=" ", header= None, index_col = 0)
val_data = pd.read_csv(os.path.join(path,"val.txt"), sep=" ", header= None, index_col = 0)

In [5]:
print("train")
# On regarde les premieres colonnes
train_data["folder"]="train"
display(train_data.head(5))

# On regarde quels sont les sources présentes dans le dataset
print(train_data[3].value_counts())


print("test")
test_data["folder"]="test"
# On regarde les premieres colonnes
display(test_data.head(5))

# On regarde quels sont les sources présentes dans le dataset
print(test_data[3].value_counts())


print("val")
val_data["folder"]="val"
# On regarde les premieres colonnes
display(val_data.head(5))

# On regarde quels sont les sources présentes dans le dataset
print(val_data[3].value_counts())



train


,1,2,3,folder
0,,,,
379,1e64990d1b40c1758a2aaa9c7f7a85_jumbo.jpeg,negative,cohen,train
379,7223b8ad031187d9a142d7f7ca02c9_jumbo.jpeg,negative,cohen,train
380,3392dc7d262e28423caca517f98c2e_jumbo.jpeg,negative,cohen,train
380,ec3a480c0926ded74429df416cfb05_jumbo.jpeg,negative,cohen,train
382,a72aeb349a63c79ed24e473c434efe_jumbo.jpg,negative,cohen,train


3
bimcv         43142
stonybrook    12696
rsna           9719
sirm            878
ricord          834
cohen           471
actmed          101
fig1             22
Name: count, dtype: int64
test


,1,2,3,folder
0,,,,
419639-003251,MIDRC-RICORD-1C-419639-003251-46647-0.png,positive,ricord,test
419639-001464,MIDRC-RICORD-1C-419639-001464-39871-0.png,positive,ricord,test
419639-000918,MIDRC-RICORD-1C-419639-000918-78965-0.png,positive,ricord,test
419639-003318,MIDRC-RICORD-1C-419639-003318-64285-0.png,positive,ricord,test
419639-001015,MIDRC-RICORD-1C-419639-001015-81591-0.png,positive,ricord,test


3
bimcv     8082
ricord     200
rsna       200
Name: count, dtype: int64
val


,1,2,3,folder
0,,,,
ANON196,CR.1.2.840.113564.1722810170.20200318082923328...,positive,actmed,val
ANON170,CR.1.2.840.113564.1722810170.20200320011259312...,positive,actmed,val
S03044,sub-S03044_ses-E06138_run-1_bp-chest_vp-ap_cr-...,positive,bimcv,val
S03044,sub-S03044_ses-E07712_run-1_bp-chest_vp-ap_cr-...,positive,bimcv,val
S03044,sub-S03044_ses-E06790_run-1_bp-chest_vp-pa_dx-...,positive,bimcv,val


3
rsna          3869
bimcv         3408
stonybrook     940
cohen           96
sirm            65
ricord          62
actmed          31
fig1             2
Name: count, dtype: int64


In [6]:
# On concatene les trois datasets pour avoir une vue d'ensemble des sources

tableau = pd.concat([train_data, test_data, val_data], axis=0)
tableau.shape

print("On compte les differentes sources de donnees")
print(tableau[3].value_counts())

print("\n")
print("On croise la source et le test COVID")
print(pd.crosstab(tableau[2],tableau[3]))

print("\n")
print("On croise la source et le folder")
print(pd.crosstab(tableau[3],tableau["folder"]))

On compte les differentes sources de donnees
3
bimcv         54632
rsna          13788
stonybrook    13636
ricord         1096
sirm            943
cohen           567
actmed          132
fig1             24
Name: count, dtype: int64


On croise la source et le test COVID
3         actmed  bimcv  cohen  fig1  ricord   rsna  sirm  stonybrook
2                                                                    
negative     107   4945    297     0       0  13788     0           0
positive      25  49687    270    24    1096      0   943       13636


On croise la source et le folder
folder      test  train   val
3                            
actmed         0    101    31
bimcv       8082  43142  3408
cohen          0    471    96
fig1           0     22     2
ricord       200    834    62
rsna         200   9719  3869
sirm           0    878    65
stonybrook     0  12696   940


### Choix des données 
Les données issues de bimcv et rnsa étaient déjà dans le dataset initial
Les données stonybrook, ricord et sirm ne contiennent que des images de patients covid+

On sélectionne donc les données actmed, cohen, et fig1  pour la suite

Dans un second temps on refait l analyse en conservant uniquement les bimcv

In [9]:
src_selec =  ["actmed", "fig1","cohen"]
src_selec = ["bimcv"]
tableau_sel  = tableau[tableau[3].isin(src_selec)]
tableau_sel.columns = ['filename', 'class','source','folder']
#tableau_sel = tableau_sel.drop("rien", axis=1)

# Remplacer des valeurs spécifiques dans la colonne 'col2'
#tableau_sel["class"] = tableau_sel["class"].replace({'positive': 'COVID', 'negative': 'Normal'})

display(tableau_sel.tail())

print("Nombres d'images sélectionnées :", tableau_sel.shape[0])

print("Occurence dans le nouveau dataset:", tableau_sel["class"].value_counts())


,filename,class,source,folder
0,,,,
S26066,sub-S26066_ses-E53803_run-1_bp-chest_vp-pa_dx-...,negative,bimcv,val
S26093,sub-S26093_ses-E53848_run-1_bp-chest_vp-pa_dx-...,negative,bimcv,val
S26141,sub-S26141_ses-E53963_run-1_bp-chest_vp-pa_dx-...,negative,bimcv,val
S26144,sub-S26144_ses-E53969_run-1_bp-chest_vp-pa_cr-...,negative,bimcv,val
S26297,sub-S26297_ses-E54337_run-1_bp-chest_vp-pa_cr-...,negative,bimcv,val


Nombres d'images sélectionnées : 54632
Occurence dans le nouveau dataset: class
positive    49687
negative     4945
Name: count, dtype: int64


In [11]:
# On exporte le fichier des données selectionnées
filepath = (os.path.join(path,"files_selectedBIMCV.csv"))
tableau_sel.to_csv(filepath, index = False)


In [12]:
# On crée un sous-répertoire

import pandas as pd
import shutil
import os

# Lire le fichier CSV contenant les noms des fichiers à copier et les sous-dossiers
file_list = tableau_sel

# Copier les fichiers dans les sous-dossiers
for index, row in file_list.iterrows():
    file_name = row['filename']  # Remplacez 'filename' par le nom de la colonne dans votre fichier CSV
    source_folder = row['folder']  # Remplacez 'subfolder' par le nom de la colonne des sous-dossiers
    destination_folder = row['class']

    source_path = os.path.join(path,source_folder, file_name)
    destination_subfolder = os.path.join(path, "BIMCV", destination_folder)
    
    # Créer le sous-dossier dans le dossier de destination s'il n'existe pas déjà
    if not os.path.exists(destination_subfolder):
        os.makedirs(destination_subfolder)
    
    destination_path = os.path.join(destination_subfolder, file_name)
    
    # Vérifier si le fichier existe dans le dossier source
    if os.path.exists(source_path):
        shutil.copy(source_path, destination_path)
        #print(f"Fichier copié : {file_name} dans le sous-dossier {destination_subfolder}")
        print()
    else:
        print(f"Fichier non trouvé : {file_name}")



In [ ]:
destination =  os.path.join(path, "SELECT","files_selected.csv")
tableau_sel.to_csv(destination, index = False)